In [1]:
import pandas as pd
a = pd.read_csv("data/metadata/train-annotations-bbox.csv", index_col=0)
b = pd.read_csv("data/metadata/train-annotations-human-imagelabels-boxable.csv", index_col=0)
c = pd.read_csv("data/metadata/train-images-ids.csv", index_col=0)

/Users/alex/miniconda3/envs/quilt-open-images-dev/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
imglist = a['ImageID'].unique()

In [7]:
len(imglist)

1743042

In [9]:
a = a.set_index("ImageID")
b = b.set_index("ImageID")
c = c.set_index("ImageID")

In [40]:
def defineAnnotationsByID(ImageID):
    bboxes   = pd.DataFrame(a.loc[ImageID])
    labels   = pd.DataFrame(b.loc[ImageID])
    img_meta = c.loc[ImageID]
    
    out = dict(img_meta)
    out['bboxes'] = list(bboxes.apply(lambda srs: dict(srs), axis='columns'))
    out['labels'] = list(labels.apply(lambda srs: dict(srs), axis='columns'))
    
    return out

In [43]:
from tqdm import tqdm_notebook

meta_entries = []
for ImageID in tqdm_notebook(imglist):
    meta_entries.append(defineAnnotationsByID(ImageID))

KeyboardInterrupt: 

This code could generate the full metadata annotations for the images as a JSON blob (though I need to also throw the image label IDs into the mix). It would take about 3 hours to generate the full dataset metadata entry in JSON.

The images can be localized by passing through the Flickr API, but the Flickr API rate limits to 3600 requests per hour (documented [here](https://www.flickr.com/services/developer/api/)). Respecting this rate limit, it would take almost 500 hours, or approximately 20 days of continuous work, in order to download the images from Flickr. Not practical.

The images could be downloaded more quickly as a direct ZIP, but I don't have enough storage space on my local drive to do it, as the zipped file is over a terabyte in size.

The best way to populate the images would be to use an Amazon EC2 instance as an intermediary, per [this comment](https://datascience.stackexchange.com/questions/5589/downloading-a-large-dataset-on-the-web-directly-into-aws-s3).

I can push the raw CSV files right away, and build some summaries around those; and do the `README.md`.

In [44]:
1743042/3600

484.17833333333334

In [16]:
# j = a.set_index("ImageID").join(b.set_index("ImageID"), lsuffix='_BBox', rsuffix='_Label')
# j.join(c.set_index("ImageID"), how='left')